In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import transformers
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [ ]:
from tqdm import tqdm

In [ ]:
test = pd.read_csv('../input/shopee-product-matching/test.csv')

In [ ]:
train_orig = pd.read_csv("../input/shopee-product-matching/train.csv") 

In [ ]:
train = pd.read_csv("../input/shopee-siamese-training/siamese_data.csv") 

In [ ]:
train.head()

In [ ]:
#model = transformers.BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)
#print(model)

In [ ]:
#ls

In [ ]:
#model.save_pretrained("./pre_bert")

In [ ]:
#model = transformers.BertTokenizer.from_pretrained("./pre_bert")

In [ ]:
#print(model)

In [ ]:
max_length = 357    #maximum length of input sentence
batch_size = 32     #number of samples propagated through the network
epochs = 2          #one epoch: entire dataset is passed forward and backward through the neural network

labels = [0, 1]     # 0:no similarity; 1:similar

data = pd.read_csv("../input/shopee-siamese-training/siamese_data.csv", nrows=10000)
train_df, test_df = train_test_split(data, test_size=0.2, shuffle=True)     #split dataset into train and test
train_df, val_df = train_test_split(train_df, test_size=0.2, shuffle=True)  # split training dataset to also get an validation dataset

print(f"Total train samples: {train_df.shape[0]}")
print(f"Total validation samples: {val_df.shape[0]}")
print(f"Total test samples: {test_df.shape[0]}")

# print(f"Title1: {train_df.loc[1, 'title_1']}")
# print(f"title2: {train_df.loc[1, 'title_2']}")
# print(f"similarity: {train_df.loc[1, 'label']}")

print("Train target distribution")
print(train_df.label.value_counts())

print("Valid target distribution")
print(val_df.label.value_counts())

print("Test target distribution")
print(test_df.label.value_counts())

y_train = tf.keras.utils.to_categorical(train_df.label, num_classes=2)
y_test = tf.keras.utils.to_categorical(test_df.label, num_classes=2)
y_val = tf.keras.utils.to_categorical(val_df.label, num_classes=2)

class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data.

    Args:
        sentence_pairs: Array of title_1 and title_2.
        labels: Array of labels.
        batch_size: Integer batch size.
        shuffle: boolean, whether to shuffle the data.
        include_targets: boolean, whether to include the labels.

    Returns:
        Tuples `([input_ids, attention_mask, `token_type_ids], labels)`
        (or just `[input_ids, attention_mask, `token_type_ids]`
         if `include_targets=False`)
    """

    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=batch_size,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load BERT Tokenizer to encode the text
        # bert-base-uncased pretrained model: pretrained model on English language
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "../input/pre-bert"
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()     #logs a dictionary with metric results for one training epoch (and validation epoch); example: {'loss':0.2, 'accuracy':0.7}

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size  # //-floor division: abrunden

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]
        
                # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,    # example: SEP, CLS
            max_length=max_length,
            return_attention_mask=True, # 1 indicates a value that should be attended to; no attention to values with 0
            return_token_type_ids=True, # first sequence(sentence) represented by 0; second sequence represented by 1
            pad_to_max_length=True,     # sentences are padded to the maximum sentence length
            truncation=True,
            return_tensors="tf",        # returns TensorFlow tf.constant object (PyTorch also possible)
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")               #tokens converted into IDs
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

    def on_epoch_end(self):
        # Shuffle indexes after each epoch if shuffle is set to True.
        if self.shuffle:
            np.random.RandomState(42).shuffle(self.indexes)
            
# Create the model under a distribution strategy scope.
strategy = tf.distribute.MirroredStrategy()     # for training with multiple GPUs

with strategy.scope():
    # Encoded token ids from BERT tokenizer.
    input_ids = tf.keras.layers.Input(                          # input is used to instantiate a Keras tensor
        shape=(max_length,), dtype=tf.int32, name="input_ids"   # shape: indicates that the expected input will be batches of max_length-dimensional vectors; name: optional unique name string
    )
    # Attention masks indicates to the model which tokens should be attended to.
    attention_masks = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="attention_masks"
    )
    # Token type ids are binary masks identifying different sequences in the model.
    token_type_ids = tf.keras.layers.Input(
        shape=(max_length,), dtype=tf.int32, name="token_type_ids"
    )
    # Loading pretrained BERT model.
    bert_model = transformers.TFBertModel.from_pretrained("../input/pre-bert")
    # Freeze the BERT model to reuse the pretrained features without modifying them.
    bert_model.trainable = False

#     sequence_output, pooled_output = bert_model(
#         input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
#     )
    bert_output = bert_model(
        input_ids, attention_mask=attention_masks, token_type_ids=token_type_ids
    )
    sequence_output = bert_output.last_hidden_state
    pooled_output = bert_output.pooler_output
    # Add trainable layers on top of frozen layers to adapt the pretrained features on the new data.
    bi_lstm = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(64, return_sequences=True) # 64: dimensionality of output space
    )(sequence_output)
    # Applying hybrid pooling approach to bi_lstm sequence output.
    avg_pool = tf.keras.layers.GlobalAveragePooling1D()(bi_lstm)
    max_pool = tf.keras.layers.GlobalMaxPooling1D()(bi_lstm)
    concat = tf.keras.layers.concatenate([avg_pool, max_pool])
    dropout = tf.keras.layers.Dropout(0.3)(concat)  # to prevent overfitting; sets randomly input units to 0 with a frequency of 0.3
    output = tf.keras.layers.Dense(2, activation="softmax")(dropout)    # dense implements output = activation(...)
    model = tf.keras.models.Model(
        inputs=[input_ids, attention_masks, token_type_ids], outputs=output
    )
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),       # optimization algorithm to update network weights iterative
        loss="categorical_crossentropy",
        metrics=["acc"],
    )

print(f"Strategy: {strategy}")
model.summary()

train_data = BertSemanticDataGenerator(
    train_df[["title_1", "title_2"]].values.astype("str"),
    y_train,
    batch_size=batch_size,
    shuffle=True,
)
valid_data = BertSemanticDataGenerator(
    val_df[["title_1", "title_2"]].values.astype("str"),
    y_val,
    batch_size=batch_size,
    shuffle=False,
)

history = model.fit(
    train_data,
    validation_data=valid_data,     # data on which to evaluate the loss and metrics at the end of each epoch
    epochs=epochs,
    use_multiprocessing=True,
    workers=-1,
)


In [ ]:
test.head()

In [ ]:
test = pd.read_csv('../input/test-100/test_100.csv')

In [ ]:
test_comp = pd.DataFrame(columns=['posting_id1', 'posting_id2', 'title1', 'title2'])
for i in tqdm(range(len(test))):
    posting1 = []
    posting2 = []
    title1 =[]
    title2 = []
    for j in range(len(test)):
        posting1.append(test['posting_id'].iloc[i])
        posting2.append(test['posting_id'].iloc[j])
        title1.append(test['title'].iloc[i])
        title2.append(test['title'].iloc[j])
    tmp_df = pd.DataFrame()
    tmp_df['posting_id1'] = posting1
    tmp_df['posting_id2'] = posting2
    tmp_df['title1'] = title1
    tmp_df['title2'] = title2
    test_comp = test_comp.append(tmp_df, ignore_index = True)

In [ ]:
def check_similarity(title_1, title_2):
    sentence_pairs = np.array([[str(title_1), str(title_2)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )

    proba = model.predict(test_data)[0]
    idx = np.argmax(proba)
    proba = f"{proba[idx]: .2f}%"
    pred = labels[idx]
    return pred, proba

prediction = []
probability = []
for i in tqdm(range(len(test_comp))):
    pred, proba = check_similarity(test_comp['title1'].iloc[i], test_comp['title2'].iloc[i])
    prediction.append(pred)
    probability.append(proba)
test_comp['prediction'] = prediction
test_comp['probability'] = probability

In [ ]:
test_comp.head()

In [ ]:
submission_df = pd.DataFrame(columns=['posting_id', 'matches'])
for i in test_comp.posting_id1.unique():
    matches = []
    id_df = test_comp[(test_comp.posting_id1 == i) & (test_comp.prediction == 1)]
    matches = id_df.posting_id2.values
    matches = np.append(matches, i)
    submission_df = submission_df.append({'posting_id':i, 'matches': matches}, ignore_index=True)
    #submission_df = submission_df.append({'posting_id':i, 'matches': ' '.join(matches)}, ignore_index=True)

In [ ]:
submission_df.head()

In [ ]:
train['prediction'] = submission_df.matches

In [ ]:
def F1_score(target_column, pred_column):
    '''Returns the F1_score for each row in the data.
    Remember: The final score is the mean F1 score.
    target_column: the name of the column that contains the target
    pred_column: the name of the column that contains the prediction
    '''
    
    def get_f1(row):
        # Find the common values in target and prediction arrays.
        intersection = len( np.intersect1d(row[target_column], row[pred_column]) )
        # Computes the score by following the formula
        f1_score = 2 * (intersection / (len(row[target_column]) + len(row[pred_column])))
        
        return f1_score
    
    return get_f1

In [ ]:
train['F1'] = train.apply(F1_score(target_column="target", pred_column="prediction"), axis=1)

In [ ]:
print('CV score for baseline = {:.3f}'.format(train["F1"].mean()))

In [ ]:
submission_df.to_csv('submission.csv', index=False)